# Imports y configuraciones iniciales

In [1]:
import warnings

from typing import List

warnings.filterwarnings("ignore")

In [2]:
# Dejar el path principal como el anterior.
import sys

sys.path.append('../')

In [3]:
import logging

# Crear el logger
log = logging.getLogger(__name__)

# Setear el nivel del registro
log.setLevel(logging.WARNING)

# Formato de los mensajes
formatter = logging.Formatter("%(levelname)s: (%(asctime)s) [%(filename)s: %(lineno)s] %(message)s")

if not log.hasHandlers():
    # Handlers
    file_handler = logging.FileHandler("logging.log")
    file_handler.setFormatter(formatter)  # Setear el formato del handler
    # Agregar el handler al logger
    # log.addHandler(file_handler)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    # Agregar el handler al logger
    log.addHandler(stream_handler)

Instalar la librería plot-likert y otras librerías útiles

In [4]:
# Librería para hacer gráficos Likert
# !pip install plot-likert

# Para obtener datos de excel
# !pip install openpyxl

# Para tener un transformador de data
# !pip install -U scikit-learn

# Para tener herramientas estadísticas
# !python -m pip install statsmodels

# Para tener Seaborn
# !pip install seaborn

Importamos algunas librerías útiles para el resto del notebook

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Importación del data set

Empezamos obteniendo los datos

In [6]:
DATA_PATH = Path(r"..\data")

if not DATA_PATH.exists():
    log.info(f"Creando carpeta {DATA_PATH = }")
    DATA_PATH.mkdir()

In [7]:
QNA_COMPLETE_PATH = DATA_PATH / "questions-and-answers-complete.xlsx"
QNA_COMPLETE_PATH

WindowsPath('../data/questions-and-answers-complete.xlsx')

In [8]:
log.debug(f"Importando datos de {QNA_COMPLETE_PATH}")
df = pd.read_excel(QNA_COMPLETE_PATH)

log.debug(f"Cantidad de respuestas: {len(df) = }")

columnas = list(df.columns)
preguntas = columnas[1:]
notas = columnas[0]

df.head()

,Prom Cs,¿Los profesores de ciencias consideran los intereses de los alumnos para elaborar y organizar las evaluaciones?,De las actividades realizadas en clase ¿el profesor considera y respeta nuestros intereses?,¿El profesor retroalimenta y refuerza en todas las clases lo que hemos ido aprendiendo?,El profesor de la clase de ciencias ¿anota y explica el objetivo que se va a trabajar durante la clase?,"El profesor, al cierre de su clase ¿realiza preguntas para verificar si hemos aprendido?",Cuando el profesor da tareas ¿las explica claramente y nos da ejemplos que orientan para realizarlas?,Durante la actividad ¿el profesor circula explicando y resolviendo dudas?,"¿Se realizan salidas a terreno: museos, zoológicos y otros lugares que ayudan al aprendizaje?","Para hacer las clases, el profesor ¿nos lleva a otros espacios motivantes para que aprendamos mejor?",...,"Cuando me queda alguna duda de la clase, ¿me dirijo confiadamente al profesor a preguntar?","Cuando el profesor es cercano conmigo y mis compañeros, ¿nos da confianza y seguridad para preguntar todas las dudas?",Cuando tengo más cercanía y confianza con el profesor ¿aprendo mejor?,"¿Los profesores de ciencias ofrecen diversas actividades grupales de un mismo tema, para que los estudiantes, elijan en la que quieren trabajar?",¿Los profesores de ciencias integran a todos los estudiantes para que participen en las actividades grupales dentro de la sala de clases?,"¿Los profesores de ciencias integran a todos los estudiantes en grupos, para que participen en las actividades de investigación, fuera del colegio?","¿Los profesores, muestran interés por reforzar a los alumnos que más les cuesta?","Cuando el profesor nos hace realizar actividades variadas, en ciencias ¿comprendo mejor la materia?",¿Los profesores de ciencias integran a todos los estudiantes para que participen en las actividades prácticas de laboratorio?,"En las clases de ciencias, ¿algunos alumnos son evaluados con carpetas de actividades realizadas en el colegio y fuera de él?"
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempr

In [9]:
questions_ = preguntas.copy()
# Separar por subdimensiones

# Subdim 1.1
subdim_1_1 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.2
subdim_1_2 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.3
subdim_1_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0)]

# Subdim 2.1
subdim_2_1 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.2
subdim_2_2 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.3
subdim_2_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 2.4
subdim_2_4 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0), questions_.pop(0)]

del questions_

In [10]:
questions = list(df.columns)[1:]
questions

['¿Los profesores de ciencias consideran los intereses de los alumnos para elaborar y organizar las evaluaciones?',
 'De las actividades realizadas en clase ¿el profesor considera y respeta nuestros intereses?',
 '¿El profesor retroalimenta y refuerza en todas las clases lo que hemos ido aprendiendo?',
 'El profesor de la clase de ciencias ¿anota y explica el objetivo que se va a trabajar durante la clase?',
 'El profesor, al cierre de su clase ¿realiza preguntas para verificar si hemos aprendido?',
 'Cuando el profesor da tareas ¿las explica claramente y nos da ejemplos que orientan para realizarlas?',
 'Durante la actividad ¿el profesor circula explicando y resolviendo dudas?',
 '¿Se realizan salidas a terreno: museos, zoológicos y otros lugares que ayudan al aprendizaje?',
 'Para hacer las clases, el profesor ¿nos lleva a otros espacios motivantes para que aprendamos mejor?',
 'El profesor de ciencias ¿presenta en sus clases material atractivo para aprender?',
 '¿Los profesores comp

In [11]:
import copy


log.debug("Renombrando columnas")
questions_copy = copy.copy(questions)
for i in range(len(questions)):
    # questions_copy[i] = f"P{i+1}.: " + questions[i]
    questions_copy[i] = f"P{i+1}"

questions = questions_copy.copy()

df.columns = ["Notas"] +  questions_copy


df.head()

,Notas,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca


In [12]:
scales = [
    "Nunca",
    "Rara vez",
    "Ocasionalmente",
    "Casi siempre",
    "Siempre, en todas las clases"
]

In [13]:
df

,Notas,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,4.8,Casi siempre,"Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Casi siempre,Casi siempre,Casi siempre,Rara vez,Casi siempre,...,Casi siempre,Casi siempre,Casi siempre,Casi siempre,Casi siempre,Casi siempre,"Siempre, en todas las clases",Casi siempre,Casi siempre,Casi siempre
82,5.7,Ocasionalmente,Rara vez,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases",Rara vez,Rara vez,Rara vez,...,Nunca,Rara vez,Rara vez,Ocasionalmente,Ocasionalmente,Ocasionalmente,Ocasionalmente,Nunca,Nunca,Nunca
83,6.6,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases",Nunca,Rara vez,...,"Siempre, en todas las clases",Casi siempre,Rara vez,Ocasionalmente,"Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca
84,6.4,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Rara vez,Rara vez,...,Casi siempre,Casi siempre,"Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente


Codificamos las preguntas para calcular el VIF

In [14]:
from collections import OrderedDict
df_enc = OrderedDict()

encoder = {name: i-2 for i, name in enumerate(scales)}
for col_name in questions_copy:
    df_enc[col_name] = df[col_name].map(encoder)

df_enc["Notas"] = (df["Notas"] - 1) / 6

df_enc = pd.DataFrame(df_enc)
df_enc

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P23,P24,P25,P26,P27,P28,P29,P30,P31,Notas
0,0,1,2,2,2,1,2,-2,-1,1,...,2,2,1,1,-1,1,2,2,-2,0.966667
1,2,2,2,2,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,0.950000
2,2,2,2,2,0,2,2,-2,0,2,...,2,2,2,2,-2,2,2,2,0,0.850000
3,1,1,-1,1,1,1,0,-2,0,0,...,1,2,0,1,2,-1,0,-1,-2,0.600000
4,2,2,2,2,2,2,1,-2,0,2,...,2,2,2,2,0,2,2,2,-2,0.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1,2,1,2,1,1,1,-1,1,2,...,1,1,1,1,1,2,1,1,1,0.633333
82,0,-1,1,2,0,2,-1,-1,-1,-2,...,-1,-1,0,0,0,0,-2,-2,-2,0.783333
83,2,0,1,2,2,0,2,-2,-1,2,...,1,-1,0,2,1,2,1,2,-2,0.933333
84,2,2,2,2,2,2,2,-1,-1,2,...,1,2,1,2,2,2,2,2,0,0.900000


Asignamos a la variable `X` las columnas a determinar si poseen algún tipo de correlación

In [15]:
X = df_enc[questions]
X

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,0,1,2,2,2,1,2,-2,-1,1,...,1,2,2,1,1,-1,1,2,2,-2
1,2,2,2,2,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,0,2,2,-2,0,2,...,2,2,2,2,2,-2,2,2,2,0
3,1,1,-1,1,1,1,0,-2,0,0,...,-2,1,2,0,1,2,-1,0,-1,-2
4,2,2,2,2,2,2,1,-2,0,2,...,2,2,2,2,2,0,2,2,2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1,2,1,2,1,1,1,-1,1,2,...,1,1,1,1,1,1,2,1,1,1
82,0,-1,1,2,0,2,-1,-1,-1,-2,...,-2,-1,-1,0,0,0,0,-2,-2,-2
83,2,0,1,2,2,0,2,-2,-1,2,...,2,1,-1,0,2,1,2,1,2,-2
84,2,2,2,2,2,2,2,-1,-1,2,...,1,1,2,1,2,2,2,2,2,0


Armamos la matriz con los valores del VIF

In [16]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

vif_data

,feature,VIF
0,P1,5.323083
1,P2,4.729647
2,P3,13.560347
3,P4,13.861366
4,P5,5.564450
5,P6,8.411208
6,P7,9.623082
7,P8,14.770040
8,P9,7.026705
9,P10,5.982414


Obtenemos aquellas preguntas que servirán para hacer la regresión

In [17]:
vif_data[vif_data["VIF"] <= 5]

,feature,VIF
1,P2,4.729647
13,P14,2.318745
14,P15,4.638542
15,P16,3.823538
16,P17,3.241056
19,P20,3.543329
24,P25,2.698775
26,P27,2.793751
29,P30,1.924325
30,P31,4.269762


In [18]:
vif_data[vif_data["VIF"] <= 12]

,feature,VIF
0,P1,5.323083
1,P2,4.729647
4,P5,5.564450
5,P6,8.411208
6,P7,9.623082
8,P9,7.026705
9,P10,5.982414
11,P12,6.937460
13,P14,2.318745
14,P15,4.638542
